어제 제공된 3개의 파일을 활용한 [실습]
- ratings.dat
- movies.dat
- users.dat


1. 각 파일을 pd.read_csv() 로 읽어들인다
 - 헤더 및 구분자 확인요망

2. 데이터 확인 및 구조분석과 각 데이터 프레임에 헤더를 추가  
ratings.dat  
 - userId movieId rating timestamp  
movies.data  
 - movieId title genres  
users.data  
 - userId gender age (뒤의 두 컬럼은 제거)  

3. 공통 컬럼을 이용한 데이터 병합

4. user 평점 갯수 순위 확인



7. 평점이 200개 이상인 영화들의 성별 평균 평점

8. 여성이 가장 좋아하는 영화 순위

9. 남녀간의 평균 평점 차이

내일 진행
5. 각 영화의 성별 평균 평점 (제외)

6. 평점이 220개 이상인 영화만 필터링 (제외)

.10. 평점의 표준편차가 큰 영화 (제외)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

1. 각 파일을 pd.read_csv() 로 읽어들인다

In [4]:
ratings = pd.read_csv('./data/ratings.dat', sep='::', header = None)
movies = pd.read_csv('./data/movies.dat', sep='::', header = None)
users = pd.read_csv('./data/users.dat', sep='::', header = None)

C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\Users\wnsgo\anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from th

2. 데이터 확인 및 구조분석과 각 데이터 프레임에 헤더를 추가   
ratings.dat  
  - userId movieId rating timestamp  
  
movies.data  
  - movieId title genres  
  
users.data  
  - userId gender age (뒤의 두 컬럼은 제거)

In [6]:
ratings.columns = ['userId', 'movieId',  'rating',  'timestamp']
ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
movies.columns = ['movieId',  'title',  'genres']
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
users = users.loc[:, :2]
users.columns = ['userId', 'gender', 'age']
users.head()

,userId,gender,age
0,1,F,1
1,2,M,56
2,3,M,25
3,4,M,45
4,5,M,25


3. 공통 컬럼을 이용한 데이터 병합

In [17]:
display(ratings.head())
display(movies.head())
display(users.head())

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


,userId,gender,age
0,1,F,1
1,2,M,56
2,3,M,25
3,4,M,45
4,5,M,25


In [20]:
movie_rating = pd.merge(movies, ratings)
print(movie_rating.shape)
movie_rating.head()

(1000209, 6)


,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [22]:
users.head()

,userId,gender,age
0,1,F,1
1,2,M,56
2,3,M,25
3,4,M,45
4,5,M,25


In [24]:
data = pd.merge(users, movie_rating)
print(data.shape)
data.head()

(1000209, 8)


,userId,gender,age,movieId,title,genres,rating,timestamp
0,1,F,1,1,Toy Story (1995),Animation|Children's|Comedy,5,978824268
1,1,F,1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,5,978824351
2,1,F,1,150,Apollo 13 (1995),Drama,5,978301777
3,1,F,1,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,4,978300760
4,1,F,1,527,Schindler's List (1993),Drama|War,5,978824195


4. user 평점 갯수 순위 확인

In [31]:
user_rank = data.groupby(data['userId']).count().reset_index()[['userId', 'title']]
user_rank

,userId,title
0,1,53
1,2,129
2,3,51
3,4,21
4,5,198
...,...,...
6035,6036,888
6036,6037,202
6037,6038,20
6038,6039,123


In [41]:
rank = user_rank.sort_values(by= 'title', ascending = False)['title'].rank(ascending = False).astype(int)
user_rank['rank'] = rank
user_rank.sort_values(by='rank').head(10)

,userId,title,rank
4168,4169,2314,1
1679,1680,1850,2
4276,4277,1743,3
1940,1941,1595,4
1180,1181,1521,5
888,889,1518,6
3617,3618,1344,7
2062,2063,1323,8
1149,1150,1302,9
1014,1015,1286,10


5. 각 영화의 성별 평균 평점

In [42]:
data.head()

,userId,gender,age,movieId,title,genres,rating,timestamp
0,1,F,1,1,Toy Story (1995),Animation|Children's|Comedy,5,978824268
1,1,F,1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,5,978824351
2,1,F,1,150,Apollo 13 (1995),Drama,5,978301777
3,1,F,1,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,4,978300760
4,1,F,1,527,Schindler's List (1993),Drama|War,5,978824195


In [52]:
len(data.title.unique())

3706

In [47]:
# pivot_table
data.pivot_table('rating', 'title', 'gender', aggfunc = 'mean')

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024
...,...,...
"Zed & Two Noughts, A (1985)",3.500000,3.380952
Zero Effect (1998),3.864407,3.723140
Zero Kelvin (Kj�rlighetens kj�tere) (1995),NaN,3.500000


6. 평점이 220개 이상인 영화만 필터링

In [64]:
data.head()

,userId,gender,age,movieId,title,genres,rating,timestamp
0,1,F,1,1,Toy Story (1995),Animation|Children's|Comedy,5,978824268
1,1,F,1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,5,978824351
2,1,F,1,150,Apollo 13 (1995),Drama,5,978301777
3,1,F,1,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,4,978300760
4,1,F,1,527,Schindler's List (1993),Drama|War,5,978824195


In [117]:
movie_count = data.groupby('title').size()
movie_count

title
$1,000,000 Duck (1971)                         37
'Night Mother (1986)                           70
'Til There Was You (1997)                      52
'burbs, The (1989)                            303
...And Justice for All (1979)                 199
                                             ... 
Zed & Two Noughts, A (1985)                    29
Zero Effect (1998)                            301
Zero Kelvin (Kj�rlighetens kj�tere) (1995)      2
Zeus and Roxanne (1997)                        23
eXistenZ (1999)                               410
Length: 3706, dtype: int64

In [157]:
over220 = movie_count[movie_count.values >= 220].index
over220

movie_count.loc[over220]

title
'burbs, The (1989)                   303
10 Things I Hate About You (1999)    700
101 Dalmatians (1961)                565
101 Dalmatians (1996)                364
12 Angry Men (1957)                  616
                                    ... 
Young Guns (1988)                    562
Young Guns II (1990)                 369
Young Sherlock Holmes (1985)         379
Zero Effect (1998)                   301
eXistenZ (1999)                      410
Length: 1332, dtype: int64

7. 평점이 200개 이상인 영화들의 성별 평균 평점

In [170]:
over200 = movie_count[movie_count >= 200].index
over200

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'Year of Living Dangerously (1982)', 'Yellow Submarine (1968)',
       'Yojimbo (1961)', 'You've Got Mail (1998)', 'Young Frankenstein (1974)',
       'Young Guns (1988)', 'Young Guns II (1990)',
       'Young Sherlock Holmes (1985)', 'Zero Effect (1998)',
       'eXistenZ (1999)'],
      dtype='object', name='title', length=1426)

In [179]:
data

,userId,gender,age,movieId,title,genres,rating,timestamp
0,1,F,1,1,Toy Story (1995),Animation|Children's|Comedy,5,978824268
1,1,F,1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,5,978824351
2,1,F,1,150,Apollo 13 (1995),Drama,5,978301777
3,1,F,1,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,4,978300760
4,1,F,1,527,Schindler's List (1993),Drama|War,5,978824195
...,...,...,...,...,...,...,...,...
1000204,6040,M,25,3683,Blood Simple (1984),Drama|Film-Noir,4,960971696
1000205,6040,M,25,3703,Mad Max 2 (a.k.a. The Road Warrior) (1981),Action|Sci-Fi,4,964828575
1000206,6040,M,25,3735,Serpico (1973),Crime|Drama,4,960971654
1000207,6040,M,25,3751,Chicken Run (2000),Animation|Children's|Comedy,4,964828782


In [180]:
title_data = data.set_index('title')
over200_df = title_data.loc[over200]
over200_df = over200_df.reset_index()
over200_df

,title,userId,gender,age,movieId,genres,rating,timestamp
0,"'burbs, The (1989)",10,F,35,2072,Comedy,4,978230923
1,"'burbs, The (1989)",22,M,18,2072,Comedy,4,978153321
2,"'burbs, The (1989)",34,F,18,2072,Comedy,2,978104190
3,"'burbs, The (1989)",92,F,18,2072,Comedy,4,986189513
4,"'burbs, The (1989)",123,M,35,2072,Comedy,2,977592663
...,...,...,...,...,...,...,...,...
855725,eXistenZ (1999),5961,F,45,2600,Action|Sci-Fi|Thriller,4,957023457
855726,eXistenZ (1999),5964,M,18,2600,Action|Sci-Fi|Thriller,5,956996314
855727,eXistenZ (1999),6001,F,25,2600,Action|Sci-Fi|Thriller,4,1004811365
855728,eXistenZ (1999),6016,M,45,2600,Action|Sci-Fi|Thriller,4,956777672


In [184]:
over200_df.groupby([ 'title', 'gender' ]).mean()[['rating']]

rating
title                             gender          
'burbs, The (1989)                F       2.793478
                                  M       2.962085
10 Things I Hate About You (1999) F       3.646552
                                  M       3.311966
101 Dalmatians (1961)             F       3.791444
...                                            ...
Young Sherlock Holmes (1985)      M       3.363344
Zero Effect (1998)                F       3.864407
                                  M       3.723140
eXistenZ (1999)                   F       3.098592
                                  M       3.289086

[2852 rows x 1 columns]

In [187]:
over200_df.pivot_table('rating', 'gender', 'title', aggfunc = 'mean').T

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


8. 여성이 가장 좋아하는 영화 순위

In [188]:
data.head()

,userId,gender,age,movieId,title,genres,rating,timestamp
0,1,F,1,1,Toy Story (1995),Animation|Children's|Comedy,5,978824268
1,1,F,1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,5,978824351
2,1,F,1,150,Apollo 13 (1995),Drama,5,978301777
3,1,F,1,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,4,978300760
4,1,F,1,527,Schindler's List (1993),Drama|War,5,978824195


In [195]:
f_data = data[data['gender'] == 'F']
f_data.groupby('title').mean()['rating']

title
$1,000,000 Duck (1971)               3.375000
'Night Mother (1986)                 3.388889
'Til There Was You (1997)            2.675676
'burbs, The (1989)                   2.793478
...And Justice for All (1979)        3.828571
                                       ...   
Your Friends and Neighbors (1998)    2.888889
Zed & Two Noughts, A (1985)          3.500000
Zero Effect (1998)                   3.864407
Zeus and Roxanne (1997)              2.777778
eXistenZ (1999)                      3.098592
Name: rating, Length: 3481, dtype: float64

In [197]:
f_rating = f_data.pivot_table('rating', 'title', aggfunc = 'mean')
f_rating

,rating
title,
"$1,000,000 Duck (1971)",3.375000
'Night Mother (1986),3.388889
'Til There Was You (1997),2.675676
"'burbs, The (1989)",2.793478
...And Justice for All (1979),3.828571
...,...
Your Friends and Neighbors (1998),2.888889
"Zed & Two Noughts, A (1985)",3.500000
Zero Effect (1998),3.864407


In [211]:
rank = f_rating.sort_values(by = 'rating', ascending = False)['rating'].rank(ascending = False, method = 'min').astype(int)
f_rating['rank'] = rank
f_rating.sort_values(by='rank').head(30)

,rating,rank
title,,
"Ballad of Narayama, The (Narayama Bushiko) (1958)",5.000000,1
Lamerica (1994),5.000000,1
Country Life (1994),5.000000,1
Belly (1998),5.000000,1
"Big Combo, The (1955)",5.000000,1
Coldblooded (1995),5.000000,1
"Saltmen of Tibet, The (1997)",5.000000,1
"Other Side of Sunday, The (S�ndagsengler) (1996)",5.000000,1
Prisoner of the Mountains (Kavkazsky Plennik) (1996),5.000000,1


9. 남녀간의 평균 평점 차이

In [216]:
data.groupby('gender').mean()[['rating']]

,rating
gender,
F,3.620366
M,3.568879


In [222]:
fm_data = data.pivot_table('rating', columns = 'gender', aggfunc = 'mean')
fm_data.reset_index(inplace = True)
fm_data['F']-fm_data['M']

0    0.051487
dtype: float64

10. 평점의 표준편차가 큰 영화 

In [223]:
data.head()

,userId,gender,age,movieId,title,genres,rating,timestamp
0,1,F,1,1,Toy Story (1995),Animation|Children's|Comedy,5,978824268
1,1,F,1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,5,978824351
2,1,F,1,150,Apollo 13 (1995),Drama,5,978301777
3,1,F,1,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,4,978300760
4,1,F,1,527,Schindler's List (1993),Drama|War,5,978824195


In [229]:
std_data = data.pivot_table('rating', 'title', aggfunc = 'std')
std_data.sort_values(by='rating', ascending = False)

,rating
title,
Foreign Student (1994),2.828427
Criminal Lovers (Les Amants Criminels) (1999),2.309401
Identification of a Woman (Identificazione di una donna) (1982),2.121320
Tokyo Fist (1995),2.121320
Better Living (1998),2.121320
...,...
Light Years (1988),0.000000
Wirey Spindell (1999),0.000000
Yankee Zulu (1994),0.000000
